# Ejercicio 6: Dense Retrieval e Introducción a FAISS

# Nombre: Nelson Casa
# Fecha: 26/11/2025

## Objetivo de la práctica

Generar embeddings con sentence-transformers (SBERT, E5), e indexar documentos con FAISS

## Parte 0: Carga del Corpus
### Actividad

1. Carga el corpus 20 Newsgroups desde sklearn.datasets.fetch_20newsgroups.
2. Limita el corpus a los primeros 2000 documentos para facilitar el procesamiento.

In [ ]:
from sklearn.datasets import fetch_20newsgroups

# 1. Carga el corpus 20 Newsgroups
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

# 2. Limita el corpus a los primeros 2000 documentos
corpus = newsgroups.data[:2000]

print(f"Corpus cargado con {len(corpus)} documentos.")
print("Primer documento del corpus (primeros 200 caracteres):\n")
print(corpus[0][:200])

Corpus cargado con 2000 documentos.
Primer documento del corpus (primeros 200 caracteres):



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However,


## Parte 2: Generación de Embeddings
### Actividad

1. Usa dos modelos de sentence-transformers. Puedes usar: `'all-MiniLM-L6-v2'` (SBERT), o `'intfloat/e5-base'` (E5). Cuando uses E5, antepon `"passage: "` a cada documento antes de codificar.
2. Genera los vectores de embeddings para todos los documentos usando el modelo seleccionado.
3. Guarda los embeddings en un array de NumPy para su posterior indexación.

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# 1. Carga el modelo 'all-MiniLM-L6-v2'
model_sbert = SentenceTransformer('all-MiniLM-L6-v2')

# 2. Genera los vectores de embeddings para todos los documentos
print("Generando embeddings con 'all-MiniLM-L6-v2'...")
embeddings_sbert = model_sbert.encode(corpus, show_progress_bar=True)

# 3. Guarda los embeddings en un array de NumPy
np.save('embeddings_sbert.npy', embeddings_sbert)

print(f"Embeddings generados con SBERT: {embeddings_sbert.shape}")

## Parte 3: Consulta
### Actividad

1. Escribe una consulta en lenguaje natural. Ejemplos:

    * "God, religion, and spirituality"
    * "space exploration"
    * "car maintenance"

2. Codifica la consulta utilizando el mismo modelo de embeddings. Cuando uses E5, antepon `"query: "` a la consulta.
3. Recupera los 5 documentos más relevantes con similitud coseno.
4. Muestra los textos de los documentos recuperados (puedes mostrar solo los primeros 500 caracteres de cada uno).

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

# 1. Escribe una consulta en lenguaje natural
query = "space exploration"

# 2. Codifica la consulta utilizando el mismo modelo de embeddings
print(f"Codificando la consulta: '{query}'...")
query_embedding = model_sbert.encode(query)

# Calcula la similitud coseno entre la consulta y todos los embeddings del corpus
# Se necesita reformar query_embedding para que sea 2D para similitud coseno
similarities = cosine_similarity(query_embedding.reshape(1, -1), embeddings_sbert)[0]

# Obtiene los índices de los 5 documentos más similares
top_5_indices = np.argsort(similarities)[::-1][:5]

print("\nDocumentos más relevantes (primeros 500 caracteres):\n")
# 4. Muestra los textos de los documentos recuperados
for i, idx in enumerate(top_5_indices):
    print(f"--- Documento {i+1} (Similitud: {similarities[idx]:.4f}) ---")
    print(corpus[idx][:500])
    print("\n")

Codificando la consulta: 'space exploration'...

Documentos más relevantes (primeros 500 caracteres):

--- Documento 1 (Similitud: 0.4991) ---
I am posting this for a friend without internet access. Please inquire
to the phone number and address listed.
---------------------------------------------------------------------

"Space: Teaching's Newest Frontier"
Sponsored by the Planetary Studies Foundation

The Planetary Studies Foundation is sponsoring a one week class for
teachers called "Space: Teaching's Newest Frontier." The class will be
held at the Sheraton Suites in Elk Grove, Illinois from June 14 through
June 18. Participants wh


--- Documento 2 (Similitud: 0.4398) ---

Well, here goes.

The first item of business is to establish the importance space life
sciences in the whole of scheme of humankind.  I mean compared
to football and baseball, the average joe schmoe doesn't seem interested
or even curious about spaceflight.  I think that this forum can
make a major change in tha